Mediapipe code to extract body coordinates

In [2]:
import cv2
import mediapipe as mp
import csv
from mediapipe.python.solutions.pose import PoseLandmark

def find_coordinates(video_name, file_name):

    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose

    # video_file = 'FinalVid.mp4'
    video_file = video_name

    cap = cv2.VideoCapture(video_file)

    f = open(file_name,'w')
    writer = csv.writer(f)
    # writer.writerow(['Frame', 'Nose_X', 'Nose_Y', 'Left_Shoulder_X', 'Left_Shoulder_Y', 'Right_Shoulder_X', 'Right_Shoulder_Y', ...])  # Add all the required column names
    writer.writerow(['Frame', 'Nose_X', 'Nose_Y', 'Left_Shoulder_X', 'Left_Shoulder_Y', 'Right_Shoulder_X', 'Right_Shoulder_Y', 'Left_Elbow_X', 'Left_Elbow_Y', 'Right_Elbow_X', 'Right_Elbow_Y', 'Left_Wrist_X', 'Left_Wrist_Y', 'Right_Wrist_X', 'Right_Wrist_Y', 'Left_Hip_X', 'Left_Hip_Y', 'Right_Hip_X', 'Right_Hip_Y', 'Left_Knee_X', 'Left_Knee_Y', 'Right_Knee_X', 'Right_Knee_Y', 'Left_Ankle_X', 'Left_Ankle_Y', 'Right_Ankle_X', 'Right_Ankle_Y'])


    with mp_pose.Pose(min_detection_confidence=0.1, min_tracking_confidence=0.1) as pose:
        frame_count = 0
        while True:
            ret, image = cap.read()
            if not ret:
                print("End of video")
                break

            frame_count += 1
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image_rgb.flags.writeable = False
            results = pose.process(image_rgb)
            image_rgb.flags.writeable = True
            image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
            
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                # row = [frame_count, 
                #        landmarks[mp_pose.PoseLandmark.NOSE].x, landmarks[mp_pose.PoseLandmark.NOSE].y,
                #        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y,
                #        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y,
                #        ...]  # Add all the required landmark coordinates in the desired order
                row = [frame_count,
                        landmarks[PoseLandmark.NOSE].x, landmarks[PoseLandmark.NOSE].y,
                        landmarks[PoseLandmark.LEFT_SHOULDER].x, landmarks[PoseLandmark.LEFT_SHOULDER].y,
                        landmarks[PoseLandmark.RIGHT_SHOULDER].x, landmarks[PoseLandmark.RIGHT_SHOULDER].y,
                        landmarks[PoseLandmark.LEFT_ELBOW].x, landmarks[PoseLandmark.LEFT_ELBOW].y,
                        landmarks[PoseLandmark.RIGHT_ELBOW].x, landmarks[PoseLandmark.RIGHT_ELBOW].y,
                        landmarks[PoseLandmark.LEFT_WRIST].x, landmarks[PoseLandmark.LEFT_WRIST].y,
                        landmarks[PoseLandmark.RIGHT_WRIST].x, landmarks[PoseLandmark.RIGHT_WRIST].y,
                        landmarks[PoseLandmark.LEFT_HIP].x, landmarks[PoseLandmark.LEFT_HIP].y,
                        landmarks[PoseLandmark.RIGHT_HIP].x, landmarks[PoseLandmark.RIGHT_HIP].y,
                        landmarks[PoseLandmark.LEFT_KNEE].x, landmarks[PoseLandmark.LEFT_KNEE].y,
                        landmarks[PoseLandmark.RIGHT_KNEE].x, landmarks[PoseLandmark.RIGHT_KNEE].y,
                        landmarks[PoseLandmark.LEFT_ANKLE].x, landmarks[PoseLandmark.LEFT_ANKLE].y,
                        landmarks[PoseLandmark.RIGHT_ANKLE].x, landmarks[PoseLandmark.RIGHT_ANKLE].y]

                writer.writerow(row)            
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(image_bgr, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                
            cv2.imshow('Pose Estimation Video', image_bgr)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()
    f.close()

# print("Enter the video name: ")
# video = input()
# print("Enter the CSV file name: ")
# file_name = input()
# find_coordinates(video, file_name)


Sequential Deep Learning Model Implementation 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from mp_2 import find_coordinates
from sklearn.metrics import mean_squared_error, r2_score

def predictor(xi, yi):
    csv_files = ['Smash1.csv', 'Smash2.csv', 'Smash3.csv', 'Smash4.csv', 'Smash5.csv']

    # Load and concatenate the data from multiple CSV files
    data = pd.DataFrame()
    for file in csv_files:
        df = pd.read_csv(file)
        data = pd.concat([data, df], ignore_index=True)

    # Create the features and labels
    X = data[xi].values.reshape(-1, 1)  # Training variable
    y = data[yi].values.reshape(-1, 1)  # Target variable

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale the features
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert the NumPy arrays to float32
    X_train_scaled = X_train_scaled.astype(np.float32)
    X_test_scaled = X_test_scaled.astype(np.float32)
    y_train = y_train.astype(np.float32)
    y_test = y_test.astype(np.float32)

    # Create the model
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(1,)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='linear'))

    # Compile the model with the Adam optimizer
    optimizer = Adam(learning_rate=0.001)  # Adjust the learning rate as needed
    model.compile(loss='mean_squared_error', optimizer=optimizer)

    # Train the model
    model.fit(X_train_scaled, y_train, epochs=10, batch_size=32)

    # Predict on test data
    y_pred = model.predict(X_test_scaled)

    # Inverse transform the scaled data to get the original values
    y_test_original = scaler.inverse_transform(y_test)
    y_pred_original = scaler.inverse_transform(y_pred)

    # Calculate Mean Squared Error (MSE) and R-squared (R2) score
    mse = mean_squared_error(y_test_original, y_pred_original)
    r2 = r2_score(y_test_original, y_pred_original)

    print("Mean Squared Error (MSE):", mse)
    print("R-squared (R2) Score:", r2)
    print("Percentage of Variance Explained by R2: {:.2f}%".format(r2 * 100))

    # Plotting predicted coordinates
    plt.figure(figsize=(8, 6))
    plt.scatter(X_test, y_pred_original, color='red', label='Predicted')
    plt.xlabel('X Coordinate Intervals')
    plt.ylabel('Predicted Y Coordinate')
    plt.title('Predicted Coordinates')
    plt.legend()
    plt.show()

    # Plotting actual coordinates
    plt.figure(figsize=(8, 6))
    plt.scatter(X_test, y_test_original, color='blue', label='Actual')
    plt.xlabel('X Coordinate Intervals')
    plt.ylabel('Actual Y Coordinate')
    plt.title('Actual Coordinates')
    plt.legend()
    plt.show()

# Example usage
print("Enter the video name: ")
video = input()
print("Enter the CSV file name: ")
file_name = input()
find_coordinates(video, file_name)

print("What do you want to train for?\n")
print("Please Enter: ")
inp = input()

match inp:
    case "Right Shoulders":
        predictor('Right_Shoulder_X', 'Right_Shoulder_Y')
    case "Left Shoulders":
        predictor('Left_Shoulder_X', 'Right_Shoulder_Y')
    case "Right Elbow":
        predictor('Right_Elbow_X', 'Right_Elbow_Y')
    case "Left Elbow":
        predictor('Left_Elbow_X', 'Right_Elbow_Y')


Random Forest Model Implementation 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from mp_2 import find_coordinates
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

# List of CSV files to load
def predictor(xi, yi):
    csv_files = ['Smash1.csv', 'Smash2.csv', 'Smash3.csv', 'Smash4.csv',
                 'Smash5.csv', 'Smash6.csv', 'Smash7.csv', 'Smash8.csv',
                 'Smash9.csv', 'Smash10.csv', 'Smash11.csv', 'Smash12.csv',
                 'Smash13.csv', 'Smash14.csv', 'Smash15.csv', 'Smash16.csv']

    # Load and concatenate the data from multiple CSV files
    data = pd.DataFrame()
    for file in csv_files:
        df = pd.read_csv(file)
        data = pd.concat([data, df], ignore_index=True)

    # Create the features and labels
    X = data[xi].values.reshape(-1, 1)  # Training variable
    y = data[yi].values.reshape(-1, 1)  # Target variable

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale the features
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Initialize the RandomForestRegressor
    model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Train the model
    model.fit(X_train_scaled, y_train)

    df1 = pd.read_csv(file_name)
    x_input = df1[xi].values.reshape(-1, 1)
    y_input = df1[yi].values.reshape(-1, 1)
    li_scaled = scaler.transform(x_input)
    predicted_values = model.predict(li_scaled)

    # Fit a quadratic curve to the predicted values using Polynomial Regression
    poly_reg = PolynomialFeatures(degree=2)
    x_input_poly = poly_reg.fit_transform(x_input)
    line_reg = LinearRegression()
    line_reg.fit(x_input_poly, predicted_values)
    line_fit = line_reg.predict(x_input_poly)

    # Plotting the predicted curve and actual values
    plt.figure(figsize=(8, 6))
    plt.plot(x_input, predicted_values, color='red', label='Predicted')
    plt.plot(x_input, line_fit, color='green', label='Curve of Best Fit')
    plt.xlabel('X Coordinate Intervals')
    plt.ylabel('Y Coordinate')
    plt.title('Predicted Coordinates')
    plt.legend()
    plt.show()

    # Fit a quadratic curve to the actual values using Polynomial Regression
    actual_x_poly = poly_reg.fit_transform(x_input)
    line_reg_actual = LinearRegression()
    line_reg_actual.fit(actual_x_poly, y_input)
    line_fit_actual = line_reg_actual.predict(actual_x_poly)

    # Plotting the actual values
    plt.figure(figsize=(8, 6))
    plt.plot(x_input, y_input, color='blue', label='Actual')
    plt.plot(x_input, line_fit_actual, color='green', label='Curve of Best Fit (Actual)')
    plt.xlabel('X Coordinate Intervals')
    plt.ylabel('Y Coordinate')
    plt.title('Actual Coordinates')
    plt.legend()
    plt.show()

    # Evaluate the model
    y_pred = model.predict(X_test_scaled)

    # Inverse transform the scaled data to get the original values
    y_test_original = scaler.inverse_transform(y_test)
    y_pred_original = scaler.inverse_transform(y_pred.reshape(-1, 1))

    # Calculate Mean Squared Error (MSE) and R-squared (R2) score
    mse = mean_squared_error(y_test_original, y_pred_original)
    r2 = r2_score(y_test_original, y_pred_original)

    mse = mean_squared_error(y_test_original, y_pred_original)

    print("Mean Squared Error (MSE):", mse)
    print("R-squared (R2) Score:", r2)
    print("Percentage of Variance Explained by R2: {:.2f}%".format(r2 * 100))


print("Enter the video name: ")
video = input()
print("Enter the CSV file name: ")
file_name = input()
find_coordinates(video, file_name)

print("What do you want to train for?\n")
print("Please Enter: ")
inp = input()

match inp:
    case "Right Shoulders":
        predictor('Right_Shoulder_X', 'Right_Shoulder_Y')
    case "Left Shoulders":
        predictor('Left_Shoulder_X', 'Right_Shoulder_Y')
    case "Right Elbow":
        predictor('Right_Elbow_X', 'Right_Elbow_Y')
    case "Left Elbow":
        predictor('Left_Elbow_X', 'Right_Elbow_Y')


Add player tracking using bounding box

In [ ]:
import cv2
import csv

def player_tracking(video_path, output_csv_path):
    # used MIL tracker
    tracker = cv2.TrackerMIL_create()

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening the video file.")
        return

    # get first frame from the video
    ret, frame = cap.read()
    if not ret:
        print("Error reading the first frame.")
        return

    # Selected player's initial bounding box
    bbox = cv2.selectROI("Select Player", frame, fromCenter=False, showCrosshair=True)
    tracker.init(frame, bbox)

    # CSV file to write player coordinates
    with open(output_csv_path, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['Player_X', 'Player_Y'])

        # processing loop for player tracking
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Update tracker to track the player in the current frame
            success, bbox = tracker.update(frame)

            if success:
                # Get the center coordinates of the bounding box
                player_x = int(bbox[0] + bbox[2] / 2)
                player_y = int(bbox[1] + bbox[3] / 2)

                # player coordinates to the CSV file
                csv_writer.writerow([player_x, player_y])

                # Draw the bounding box and player's position on the frame
                cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3])), (0, 255, 0), 2)
                cv2.circle(frame, (player_x, player_y), 5, (0, 0, 255), -1)

            # Display current frame with player tracking
            cv2.imshow("Player Tracking", frame)

            # Press 'q' to exit 
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

video_path = "match.mp4"
output_csv_path = "player_track.csv"

# Start player tracking
player_tracking(video_path, output_csv_path)

YOLO Code to calculate the total time of the player in the frame

In [ ]:
import argparse
from pathlib import Path
import cv2
import torch
from boxmot.tracker_zoo import create_tracker
from boxmot.utils import ROOT, WEIGHTS
from boxmot.utils import logger as LOGGER
from boxmot.utils.checks import TestRequirements
from boxmot.utils.torch_utils import select_device
from detectors.strategy import get_yolo_inferer
from detectors.yolo_processor import Yolo
from ultralytics.yolo.data.utils import VID_FORMATS
from ultralytics.yolo.engine.model import TASK_MAP, YOLO
from ultralytics.yolo.utils import IterableSimpleNamespace, colorstr, ops
from ultralytics.yolo.utils.checks import check_imgsz
from ultralytics.yolo.utils.files import increment_path
from ultralytics.yolo.utils.plotting import save_one_box
from utils import write_MOT_results
from boxmot.utils import EXAMPLES


def on_predict_start(predictor):
    predictor.trackers = []
    predictor.tracker_outputs = [None] * predictor.dataset.bs
    predictor.args.tracking_config = \
        ROOT /\
        'boxmot' /\
        'configs' /\
        (opt.tracking_method + '.yaml')
    for i in range(predictor.dataset.bs):
        tracker = create_tracker(
            predictor.args.tracking_method,
            predictor.args.tracking_config,
            predictor.args.reid_model,
            predictor.device,
            predictor.args.half,
            predictor.args.per_class
        )
        predictor.trackers.append(tracker)


@torch.no_grad()
def run(args):

    model = YOLO(args['yolo_model'] if 'v8' in str(args['yolo_model']) else 'yolov8n')
    overrides = model.overrides.copy()
    model.predictor = TASK_MAP[model.task][3](overrides=overrides, _callbacks=model.callbacks)

    # extract task predictor
    predictor = model.predictor

    # combine default predictor args with custom, preferring custom
    combined_args = {**predictor.args.__dict__, **args}
    # overwrite default args
    predictor.args = IterableSimpleNamespace(**combined_args)
    predictor.args.device = select_device(args['device'])
    LOGGER.info(args)

    # setup source and model
    if not predictor.model:
        predictor.setup_model(model=model.model, verbose=False)
    predictor.setup_source(predictor.args.source)

    predictor.args.imgsz = check_imgsz(predictor.args.imgsz, stride=model.model.stride, min_dim=2)  # check image size
    predictor.save_dir = increment_path(Path(predictor.args.project) /
                                        predictor.args.name, exist_ok=predictor.args.exist_ok)

    # Check if save_dir/ label file exists
    if predictor.args.save or predictor.args.save_txt:
        (predictor.save_dir / 'labels' if predictor.args.save_txt
         else predictor.save_dir).mkdir(parents=True, exist_ok=True)

    # Warmup model
    if not predictor.done_warmup:
        predictor.model.warmup(
            imgsz=(1 if predictor.model.pt or predictor.model.triton else predictor.dataset.bs, 3, *predictor.imgsz)
        )
        predictor.done_warmup = True
    predictor.seen, predictor.windows, predictor.batch, predictor.profilers = (
        0,
        [],
        None,
        (ops.Profile(), ops.Profile(), ops.Profile(), ops.Profile())
    )
    predictor.add_callback('on_predict_start', on_predict_start)
    predictor.run_callbacks('on_predict_start')

    yolo_strategy = get_yolo_inferer(args['yolo_model'])
    yolo_strategy = yolo_strategy(
        model=model.predictor.model if 'v8' in str(args['yolo_model']) else args['yolo_model'],
        device=predictor.device,
        args=predictor.args
    )
    model = Yolo(yolo_strategy)

    # Initialize variables to keep track of person time
    person_time = {}
    person_in_frame = {}

    for frame_idx, batch in enumerate(predictor.dataset):
        predictor.run_callbacks('on_predict_batch_start')
        predictor.batch = batch
        path, im0s, vid_cap, s = batch

        n = len(im0s)
        predictor.results = [None] * n

        # Preprocess
        with predictor.profilers[0]:
            im = predictor.preprocess(im0s)

        # Inference
        with predictor.profilers[1]:
            preds = model.inference(im=im)

        # Postprocess moved to MultiYolo
        with predictor.profilers[2]:
            predictor.results = model.postprocess(path, preds, im, im0s, predictor)
        predictor.run_callbacks('on_predict_postprocess_end')

        # Visualize, save, write results
        n = len(im0s)
        for i in range(n):

            if predictor.dataset.source_type.tensor:  # skip write, show and plot operations if input is raw tensor
                continue
            p, im0 = path[i], im0s[i].copy()
            p = Path(p)

            with predictor.profilers[3]:
                # get raw bboxes tensor
                dets = predictor.results[i].boxes.data
                # get tracker predictions
                predictor.tracker_outputs[i] = predictor.trackers[i].update(dets.cpu().detach().numpy(), im0)
            predictor.results[i].speed = {
                'preprocess': predictor.profilers[0].dt * 1E3 / n,
                'inference': predictor.profilers[1].dt * 1E3 / n,
                'postprocess': predictor.profilers[2].dt * 1E3 / n,
                'tracking': predictor.profilers[3].dt * 1E3 / n
            }

            # filter boxes masks and pose results by tracking results
            model.filter_results(i, predictor)
            # overwrite bbox results with tracker predictions
            model.overwrite_results(i, im0.shape[:2], predictor)

            # write inference results to a file or directory
            if (predictor.args.verbose or predictor.args.save or
               predictor.args.save_txt or predictor.args.show or
               predictor.args.save_id_crops):

                s += predictor.write_results(i, predictor.results, (p, im, im0))
                predictor.txt_path = Path(predictor.txt_path)

                # write MOT specific results
                if predictor.args.source.endswith(VID_FORMATS):
                    predictor.MOT_txt_path = predictor.txt_path.parent / p.stem
                else:
                    # append folder name containing current img
                    predictor.MOT_txt_path = predictor.txt_path.parent / p.parent.name

                if predictor.tracker_outputs[i].size != 0 and predictor.args.save_mot:
                    write_MOT_results(
                        predictor.MOT_txt_path,
                        predictor.results[i],
                        frame_idx,
                        i,
                    )

                if predictor.args.save_id_crops:
                    for d in predictor.results[i].boxes:
                        save_one_box(
                            d.xyxy,
                            im0.copy(),
                            file=increment_path(
                                Path(predictor.save_dir) / 'id_crops' / f"{i:05d}",
                                mkdir=True
                            ) / f"{frame_idx:05d}.jpg",
                            labels=[f"{predictor.names[int(d.id)]} {d.id:.2f}"],
                            colors=[(d.color if predictor.args.save_rgb else d.color_bw)[[2, 1, 0]]]
                        )

            # display an image in a window using OpenCV imshow()
            if predictor.args.show and predictor.plotted_img is not None:
                predictor.show(p.parent)

            # save video predictions
            if predictor.args.save and predictor.plotted_img is not None:
                predictor.save_preds(vid_cap, i, str(predictor.save_dir / p.name))

            # Record person time
            for d in predictor.results[i].boxes:
                if d.cls == 0 and d.id >= 0:  # Person class and valid ID
                    person_id = int(d.id)
                    if person_id not in person_in_frame:  # Person not encountered before
                        person_in_frame[person_id] = frame_idx
                    else:  # Person already encountered, update the total time
                        person_time[person_id] = frame_idx - person_in_frame[person_id]

        predictor.run_callbacks('on_predict_batch_end')

    # Print total person time
    LOGGER.info("Total Person Time:")
    for person_id, time in person_time.items():
        frame_rate = vid_cap.get(cv2.CAP_PROP_FPS) if vid_cap.get(cv2.CAP_PROP_FPS) > 0 else 30  # Set default frame rate
        LOGGER.info(f"Person {person_id}: {time} frames ({time / frame_rate:.2f} seconds)")


def parse_opt():
    parser = argparse.ArgumentParser()
    parser.add_argument('--yolo-model', type=Path, default=WEIGHTS / 'yolov8n.pt', help='model.pt path(s)')
    parser.add_argument('--reid-model', type=Path, default=WEIGHTS / 'mobilenetv2_x1_4_dukemtmcreid.pt')
    parser.add_argument('--tracking-method', type=str, default='deepocsort',
                        help='deepocsort, botsort, strongsort, ocsort, bytetrack')
    parser.add_argument('--source', type=str, default='0',
                        help='file/dir/URL/glob, 0 for webcam')
    parser.add_argument('--imgsz', '--img', '--img-size', nargs='+', type=int, default=[640],
                        help='inference size h,w')
    parser.add_argument('--conf', type=float, default=0.5,
                        help='confidence threshold')
    parser.add_argument('--iou', type=float, default=0.7,
                        help='intersection over union (IoU) threshold for NMS')
    parser.add_argument('--device', default='',
                        help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--show', action='store_true',
                        help='display tracking video results')
    parser.add_argument('--save', action='store_true',
                        help='save video tracking results')
    # # class 0 is person, 1 is bycicle, 2 is car... 79 is oven
    parser.add_argument('--classes', nargs='+', type=int,
                        help='filter by class: --classes 0, or --classes 0 2 3')
    parser.add_argument('--project', default=EXAMPLES / 'runs' / 'track',
                        help='save results to project/name')
    parser.add_argument('--name', default='exp',
                        help='save results to project/name')
    parser.add_argument('--exist-ok', action='store_true',
                        help='existing project/name ok, do not increment')
    parser.add_argument('--half', action='store_true',
                        help='use FP16 half-precision inference')
    parser.add_argument('--vid-stride', type=int, default=1,
                        help='video frame-rate stride')
    parser.add_argument('--show-labels', action='store_false',
                        help='hide labels when show')
    parser.add_argument('--show-conf', action='store_false',
                        help='hide confidences when show')
    parser.add_argument('--save-txt', action='store_true',
                        help='save tracking results in a txt file')
    parser.add_argument('--save-id-crops', action='store_true',
                        help='save each crop to its respective id folder')
    parser.add_argument('--save-mot', action='store_true',
                        help='save tracking results in a single txt file')
    parser.add_argument('--line-width', default=None, type=int,
                        help='The line width of the bounding boxes. If None, it is scaled to the image size.')
    parser.add_argument('--per-class', action='store_true',
                        help='not mix up classes when tracking')

    opt = parser.parse_args()
    return opt


def main(opt):
    run(vars(opt))


if __name__ == "__main__":
    opt = parse_opt()
    main(opt)